In [1]:
import pandas as pd
import numpy as np
import json
import pathlib
pd.options.plotting.backend = "plotly"

MEAN_WINDOW_SIZE = 200
FIG_WIDTH= 1000
FIG_HEIGHT = 600

ENABLE_SAVE_IMAGE = True

savepath = pathlib.Path.cwd() / 'hahaha'
savepath.mkdir(parents=True, exist_ok=True)

In [2]:
# Lookup model to index for convenience plot

lookup = {
    'mobilenet_34.67_66.84': 0,
    'mobilenet_36.04_37.97': 1,
    'mobilenet_41.75_33.35': 2,
    'resnet18_39.52_66.54': 3,
    'resnet18_40.87_46.49': 4,
    'resnet18_42.63_15.38': 5,
    'resnet18_58.17_71.87': 6,
    'resnet18_58.66_55.21': 7,
}

file1 = open( "models_name_lookup.txt", "w")
for key, val in lookup.items():
    file1.write(key + ' : ' + str(val) + "\n")
file1.close()

In [3]:
# Load data from json

data = json.load(open('profiler_dump_multiple.json'))
keys = [k for k in data.keys()]

In [4]:
# remove invalid filename characters
def slugify(value):
    return "".join(x for x in value if x.isalnum())

## Mean Accuracy Plot

In [5]:
accumalted_scores = []
selected_record = data['selected_record']
for i in range(len(selected_record)):
    accumalted_scores.append(selected_record[i][-2])
df = pd.DataFrame(accumalted_scores, index=range(len(accumalted_scores)), columns =['y'])
df.index.name = 'Time'
y_name = 'Mean accuracy (%)'
df[y_name] = df.rolling(window=MEAN_WINDOW_SIZE).mean()
fig = df.reset_index().plot.line(x='Time', y=y_name, width=FIG_WIDTH, height=FIG_HEIGHT)
if ENABLE_SAVE_IMAGE: fig.write_image( str(savepath / (slugify(y_name) + ".png")))
else: fig.show()

## Mean Contol elapsed Time Plot

In [6]:
load_model_ms = data['load_model_ms']
iter_ms = data['iter_ms']

contol_elapsed_over_t = []
model_over_t = []

for i in range(len(selected_record)):
    inference_ms = selected_record[i][-1]
    contol_elapsed_over_t.append(iter_ms[i] - load_model_ms[i] - inference_ms)
    model_over_t.append(lookup[selected_record[i][0]])

    
fps_over_t = []
for i in range(len(iter_ms)):
    fps_over_t.append(1000/iter_ms[i])

In [7]:
y_name = 'Mean Control Elapsed Time (ms)'
df = pd.DataFrame(contol_elapsed_over_t, index=range(len(contol_elapsed_over_t)), columns =['y'])
df.index.name = 'Time'
df[y_name] = df.rolling(window=MEAN_WINDOW_SIZE).mean()
fig = df.reset_index().plot.line(x='Time', y=y_name, width=FIG_WIDTH, height=FIG_HEIGHT)
#fig.update_yaxes(range=[0, 3])
if ENABLE_SAVE_IMAGE: fig.write_image( str(savepath / (slugify(y_name) + ".png")))
else: fig.show()

## Mean FPS Plot

In [8]:
y_name = 'FPS'
df = pd.DataFrame(fps_over_t, index=range(len(fps_over_t)), columns =['y'])
df.index.name = 'Time'
df[y_name] = df.rolling(window=MEAN_WINDOW_SIZE).mean()
fig = df.reset_index().plot.line(x='Time', y=y_name, width=FIG_WIDTH, height=FIG_HEIGHT)
#fig.update_yaxes(range=[0, 80])
if ENABLE_SAVE_IMAGE: fig.write_image( str(savepath / (slugify(y_name) + ".png")))
else: fig.show()

In [9]:
y_name = 'Load Model elapsed time (ms)'
df = pd.DataFrame(load_model_ms, index=range(len(load_model_ms)), columns =['y'])
df.index.name = 'Time'
#df[y_name] = df.rolling(window=1).mean()
fig = df.reset_index().plot.line(x='Time', y='y', width=FIG_WIDTH, height=FIG_HEIGHT)
#fig.update_yaxes(range=[0, 80])
if ENABLE_SAVE_IMAGE: fig.write_image( str(savepath / (slugify(y_name) + ".png")))
else: fig.show()

In [10]:

y_name = 'Selected Model'
df = pd.DataFrame(model_over_t, index=range(len(model_over_t)), columns =['y'])
df.index.name = 'Time'
#df[y_name] = df.rolling(window=1).mean()
fig = df.reset_index().plot.scatter(x='Time', y='y', width=FIG_WIDTH, height=FIG_HEIGHT)
#fig.update_yaxes(range=[0, 80])
if ENABLE_SAVE_IMAGE: fig.write_image( str(savepath / (slugify(y_name) + ".png")))
else: fig.show()
